In [1]:
import pandas as pd
import duckdb
import json
import warnings
import ast

In [2]:
proposals_data_path = 'data/01-cws_proposals_data.json'
with open(proposals_data_path) as data:
    proposals = json.load(data)
    
error_log_path = 'data/02-cws_proposals_error_log.json'
with open(error_log_path) as data:
    errors = json.load(data)
    
user_data = pd.read_csv('secrets/cws_user_data_merged.tsv', sep='\t', index_col=0)

# Processing functions

In [3]:
def process_proposals(data):
    rows = []
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal in proposals:
                rows.append({'year': year, 'category': category, 'proposal': proposal})
    
    return pd.DataFrame(rows)

def process_participant_roles(data):
    rows = []
    
    roles_mapping = {
        'proposer': 'proposer',
        'discussion_participants': 'discussant',
        'voters': 'voter'
    }
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal, details in proposals.items():
                for role_key, role_singular in roles_mapping.items():
                    if role_key in details and details[role_key]:
                        rows.append({
                            'year': year,
                            'wish': proposal,
                            'role': role_singular,
                            'username': details[role_key]
                        })
    
    df = pd.DataFrame(rows)
    df = df.explode('username', ignore_index=True)
    
    return df

def process_phab_tickets(data):
    rows = []
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal, details in proposals.items():
                rows.append({
                    'year': year,
                    'proposal': proposal,
                    'phab_tickets': details.get('phab_tickets', [])
                })
    
    return pd.DataFrame(rows)

def process_user_data(df):
    df['edit_count'] = df.apply(lambda row: {'hw': row['hw_edit_count'], 'mw': row['mw_edit_count']}, axis=1)
    df['edit_bucket'] = df.apply(lambda row: {'hw': row['hw_edit_bucket'], 'mw': row['mw_bucket']}, axis=1)
    df['account_age'] = df.apply(lambda row: {'months': row['account_age_months'], 'years': row['account_age_years']}, axis=1)

    df = df.drop(columns=['hw_edit_count', 'mw_edit_count', 'hw_edit_bucket', 'mw_bucket', 'account_age_months', 'account_age_years'])
    
    return df

def process_rejects(data):
    rows = []
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal, details in proposals.items():
                try:
                    rows.append({
                        'year': year,
                        'proposal': proposal,
                        'reason': details.get('reject_reason', 0)
                    })
                except:
                    pass
    
    return pd.DataFrame(rows).query("""reason != 0""").reset_index(drop=True)

def process_error_log(data):
    rows = []

    for year, proposals in data.items():
        for proposal_name, details in proposals.items():
            category = details['category']
            error = details['error']
            rows.append({'year': year, 
                         'proposal_name': proposal_name, 
                         'category': category, 
                         'error': error})
                
    return pd.DataFrame(rows)

# Write to DB

In [4]:
conn = duckdb.connect('secrets/cws_data.db')

In [5]:
categories_mapping = {
    'Admins_and_stewards': 'Moderation_tools', 
    'Admins_and_patrollers': 'Moderation_tools',
    'Moderation_and_admin_tools': 'Moderation_tools',
    'Commons': 'Multimedia_and_Commons',
    'Multimedia': 'Multimedia_and_Commons',
    'Notifications,_Watchlists_and_Talk_Pages': 'Communication_tools',
    'Notifications': 'Communication_tools',
    'Talk_pages': 'Communication_tools',
    'Special_pages': 'Communication_tools',
    'Search_and_Categories': 'Navigation_tools',
    'Search': 'Navigation_tools',
    'Categories': 'Navigation_tools'
    }

In [6]:
proposals_by_year = process_proposals(proposals).replace(dict(category = categories_mapping))

conn.execute("""
CREATE OR REPLACE TABLE proposals AS 
SELECT
    year::INTEGER AS year,
    category,
    proposal
FROM
    proposals_by_year
""")

In [7]:
participant_roles = process_participant_roles(proposals)

conn.execute("""
CREATE OR REPLACE TABLE roles AS 
SELECT
    year::INTEGER AS year,
    wish AS proposal,
    role,
    username
FROM
    participant_roles
""")

In [8]:
phab_tickets = process_phab_tickets(proposals)

conn.execute("""
CREATE OR REPLACE TABLE phab_tickets AS 
SELECT
    year::INTEGER AS year,
    proposal,
    phab_tickets
FROM
    phab_tickets
""")

In [9]:
def convert_to_list(s):
    if pd.isna(s) or s == 'None':
        return []
    try:
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        return []

In [10]:
user_data['hw_user_groups'] = user_data['hw_user_groups'].apply(convert_to_list)
user_data_processed = process_user_data(user_data)

conn.execute("""
CREATE OR REPLACE TABLE user_data AS 
SELECT
    survey_year::INTEGER AS year,
    username,
    home_wiki,
    hw_user_groups,
    edit_count,
    edit_bucket,
    account_age
FROM
    user_data_processed
""")

In [11]:
rejected_proposals = process_rejects(proposals)

conn.execute("""
CREATE OR REPLACE TABLE rejects AS 
SELECT
    year::INTEGER AS year,
    proposal,
    reason
FROM
    rejected_proposals
""")

In [12]:
error_log = process_error_log(errors).replace(dict(category = categories_mapping))

conn.execute("""
CREATE OR REPLACE TABLE errors AS 
SELECT
    year::INTEGER AS year,
    category,
    proposal_name AS proposal,
    error
FROM
    error_log
""")

In [13]:
cd_wikis = pd.read_csv('https://raw.githubusercontent.com/wikimedia-research/canonical-data/master/wiki/wikis.tsv', sep='\t')
wiki_comparision = (pd
                    .read_csv('https://raw.githubusercontent.com/wikimedia-research/wiki-comparison/main/data-collection/snapshots/Jan_2023.tsv', sep='\t')
                    .rename({'database code': 'wiki_db'}, axis=1))

conn.execute("""
CREATE OR REPLACE TABLE cdw AS 
SELECT
    cd_wikis.database_code AS wiki_db,
    database_group AS db_group,
    language_name AS lang_name,
    english_name AS wiki_name,
    "overall size rank" AS size_rank
FROM
    cd_wikis 
    LEFT JOIN wiki_comparision wc ON cd_wikis.database_code = wc.wiki_db
""")

# Check data types

In [14]:
conn.sql("""DESCRIBE proposals""").show()
conn.sql("""DESCRIBE roles""").show()
conn.sql("""DESCRIBE phab_tickets""").show()
conn.sql("""DESCRIBE user_data""").show()
conn.sql("""DESCRIBE rejects""").show()
conn.sql("""DESCRIBE errors""").show()
conn.sql("""DESCRIBE cdw""").show()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year        │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ category    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ proposal    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year        │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ proposal    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ role        │ VARCHAR     │ YES     │ NULL   

In [17]:
conn.close()